In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from keras_tuner import HyperParameters

import autokeras as ak

from cerebro.nas.hphpmodel import HyperHyperModel

In [2]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.ClassificationHead()(output_node)
am = HyperHyperModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)

In [3]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .appName("Iris test") \
   .getOrCreate()

sc = spark.sparkContext

from cerebro.backend import SparkBackend
from cerebro.storage import LocalStore

backend = SparkBackend(spark_context=sc, num_workers=1)
store = LocalStore(prefix_path='/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments')

am.resource_bind(
    backend=backend, 
    store=store,
    feature_columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'],
    label_columns=['Species']
)

21/11/20 00:57:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


CEREBRO => Time: 2021-11-20 00:57:51, Running 1 Workers


In [4]:
df = spark.read.csv("/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/Iris_clean.csv", header=True)

train_df, test_df = df.randomSplit([0.8, 0.2])
df.head(10)

[Row(SepalLengthCm='5.1', SepalWidthCm='3.5', PetalLengthCm='1.4', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.9', SepalWidthCm='3.0', PetalLengthCm='1.4', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.7', SepalWidthCm='3.2', PetalLengthCm='1.3', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.6', SepalWidthCm='3.1', PetalLengthCm='1.5', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='5.0', SepalWidthCm='3.6', PetalLengthCm='1.4', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='5.4', SepalWidthCm='3.9', PetalLengthCm='1.7', PetalWidthCm='0.4', Species='0'),
 Row(SepalLengthCm='4.6', SepalWidthCm='3.4', PetalLengthCm='1.4', PetalWidthCm='0.3', Species='0'),
 Row(SepalLengthCm='5.0', SepalWidthCm='3.4', PetalLengthCm='1.5', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.4', SepalWidthCm='2.9', PetalLengthCm='1.4', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.9', SepalWidthCm='3.1', PetalLengthCm='1.5', PetalWidthCm='0.1', Spec

In [5]:
am.tuner_bind(tuner="randomsearch", hyperparameters=None)
am.fit(train_df, epochs=10)

INFO:tensorflow:Reloading Oracle from existing project ./test/oracle.json
CEREBRO => Time: 2021-11-20 00:57:54, Preparing Data
CEREBRO => Time: 2021-11-20 00:57:55, Num Partitions: 1
CEREBRO => Time: 2021-11-20 00:57:55, Writing DataFrames
CEREBRO => Time: 2021-11-20 00:57:55, Train Data Path: file:///Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments/intermediate_train_data
CEREBRO => Time: 2021-11-20 00:57:55, Val Data Path: file:///Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments/intermediate_val_data


CEREBRO => Time: 2021-11-20 00:57:56, Train Partitions: 1


[Stage 4:>                                                          (0 + 1) / 1]21/11/20 00:57:58 ERROR Utils: Aborting task
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/zijian/Desktop/ucsd/cse234/project/dev/Apache-Spark/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/Users/zijian/Desktop/ucsd/cse234/project/dev/Apache-Spark/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/zijian/Desktop/ucsd/cse234/project/dev/Apache-Spark/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/zijian/Desktop/ucsd/cse234/project/dev/Apache-Spark/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File

Py4JJavaError: An error occurred while calling o103.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope